In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..', '..', 'src'))
from sefl_analysis_tools import AnalysisTools
import sefl_analysis_tools as sat
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('All_cohorts_freezing_workbook_10_22_24.csv')
data = data[data['cohort'].isin(['ptsd2', 'ptsd3', 'ptsd4', 'ptsd5', 'ptsd6', 'ptsd9'])]
data_old = pd.read_csv('PTSD_filtered.csv')


In [3]:
merged_data = pd.merge(data_old, data[['id', 'cohort', 'freezing_1_75s', 'day']], on=['id', 'cohort', 'day'], how='left')
columns = ['freezing', 'freezing_1_75s'] + [col for col in merged_data.columns if col not in ['freezing', 'freezing_1_75s']]
merged_data = merged_data[columns]
merged_data = sat.calculate_age_at_sefla(merged_data)
merged_data.head()

c:\Users\Chehaoy\OneDrive - New York State Office of Information Technology Services\Documents\els_project\notebooks\novak_analyses\..\..\src\sefl_analysis_tools.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['dob'] = pd.to_datetime(data['dob'])
c:\Users\Chehaoy\OneDrive - New York State Office of Information Technology Services\Documents\els_project\notebooks\novak_analyses\..\..\src\sefl_analysis_tools.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['date'] = pd.to_datetime(data['date'])


,freezing,freezing_1_75s,genotype,id,day,condition,sex,cohort,exposure_mins,analyzed_duration,date,time,els,psilocybin,cohort_id,dob,age_at_sefla,young
0,5.03,0.90,wt,81,sefla,control,male,ptsd2,60.0,60.0,2021-04-28,11:59 AM,n,n,ptsd2_81,2021-02-16,10.142857,True
1,0.45,0.00,wt,81,seflb,control,male,ptsd2,3.5,3.5,2021-04-29,4:40 PM,n,n,ptsd2_81,2021-02-16,10.142857,True
2,8.28,0.62,wt,81,recall1,control,male,ptsd2,5.0,5.0,2021-04-30,1:37 PM,n,n,ptsd2_81,2021-02-16,10.142857,True
3,7.12,0.71,wt,81,recall2,control,male,ptsd2,5.0,5.0,2021-05-03,10:17 AM,n,n,ptsd2_81,2021-02-16,10.142857,True
4,7.57,0.00,wt,81,recall3,control,male,ptsd2,5.0,5.0,2021-05-05,10:04 AM,n,n,ptsd2_81,2021-02-16,10.142857,True


In [4]:
tools = AnalysisTools(merged_data)

In [5]:
result = tools.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='condition',
    subject='cohort_id'
)
print("Repeated ANOVA Result for conditional comparison:")
print(result['anova_result'])
print(f"Required Sample Size per Group: {result['required_sample_size']:.4f}")

Repeated ANOVA Result for conditional comparison:
        Source            SS  DF1  DF2            MS          F         p-unc  \
0    condition  22412.005695    1   67  22412.005695  36.213495  8.261861e-08   
1          day  13740.499208    4  268   3435.124802  26.509516  1.549818e-18   
2  Interaction   1853.800016    4  268    463.450004   3.576532  7.330114e-03   

      p-GG-corr       np2       eps sphericity   W-spher   p-spher  
0           NaN  0.350860       NaN        NaN       NaN       NaN  
1  2.967943e-14  0.283495  0.761674      False  0.541009  0.000005  
2           NaN  0.050676       NaN        NaN       NaN       NaN  
Required Sample Size per Group: 8.3398


In [6]:
results2 = tools.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='young',
    subject='cohort_id'
)

print("Repeated ANOVA Result for age comparison:")
print(results2['anova_result'])
print(f"Required Sample Size per Group: {results2['required_sample_size']:.4f}")


Repeated ANOVA Result for age comparison:
        Source            SS  DF1  DF2            MS          F         p-unc  \
0        young  12092.593867    1   67  12092.593867  15.645610  1.868885e-04   
1          day  13740.499208    4  268   3435.124802  26.521339  1.524257e-18   
2  Interaction   1869.281222    4  268    467.320306   3.608009  6.955236e-03   

      p-GG-corr       np2       eps sphericity   W-spher   p-spher  
0           NaN  0.189310       NaN        NaN       NaN       NaN  
1  2.967943e-14  0.283586  0.761674      False  0.541009  0.000005  
2           NaN  0.051099       NaN        NaN       NaN       NaN  
Required Sample Size per Group: 17.8191


In [7]:
sefl_merged_data = merged_data[merged_data['condition'] == 'sefl']
control_merged_data = merged_data[merged_data['condition'] == 'control']

tools3 = AnalysisTools(sefl_merged_data)
tools4 = AnalysisTools(control_merged_data)

results3 = tools3.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='young',
    subject='cohort_id'
)

print("Repeated ANOVA Result for age comparison in SEFL:")
print(results3['anova_result'])
print(f"Required Sample Size per Group: {results3['required_sample_size']:.4f}")


results4 = tools4.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='young',
    subject='cohort_id'
)

print("Repeated ANOVA Result for age comparison in control:")
print(results4['anova_result'])
print(f"Required Sample Size per Group: {results4['required_sample_size']:.4f}")


Repeated ANOVA Result for age comparison in SEFL:
        Source            SS  DF1  DF2            MS          F         p-unc  \
0        young  13621.578445    1   41  13621.578445  22.313943  2.714732e-05   
1          day  13683.867112    4  164   3420.966778  19.460713  4.028269e-13   
2  Interaction   1423.287725    4  164    355.821931   2.024150  9.339234e-02   

      p-GG-corr       np2      eps sphericity   W-spher   p-spher  
0           NaN  0.352433      NaN        NaN       NaN       NaN  
1  1.540821e-10  0.321874  0.77897      False  0.530311  0.002362  
2           NaN  0.047047      NaN        NaN       NaN       NaN  
Required Sample Size per Group: 8.2906
Repeated ANOVA Result for age comparison in control:
        Source           SS  DF1  DF2          MS          F         p-unc  \
0        young   594.730511    1   24  594.730511   6.428047  1.816246e-02   
1          day  1910.432112    4   96  477.608028  11.911890  6.716479e-08   
2  Interaction   625.954961

In [8]:
young_merged_data = merged_data[merged_data['young'] == 'True']
old_merged_data = merged_data[merged_data['young'] == 'False']

tools5 = AnalysisTools(young_merged_data)
tools6 = AnalysisTools(old_merged_data)

results5 = tools5.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='condition',
    subject='cohort_id'
)

print("Repeated ANOVA Result for condition comparison in young adult mice:")
print(results5['anova_result'])
print(f"Required Sample Size per Group: {results5['required_sample_size']:.4f}")


results6 = tools6.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='condition',
    subject='cohort_id'
)

print("Repeated ANOVA Result for condition comparison in adult mice:")
print(results6['anova_result'])
print(f"Required Sample Size per Group: {results6['required_sample_size']:.4f}")


Repeated ANOVA Result for condition comparison in young adult mice:
        Source           SS  DF1  DF2           MS          F         p-unc  \
0    condition  5979.262283    1   37  5979.262283  17.387539  1.763262e-04   
1          day  4456.775034    4  148  1114.193758  12.991204  4.313257e-09   
2  Interaction  1225.047135    4  148   306.261784   3.570931  8.219542e-03   

   p-GG-corr       np2       eps sphericity  W-spher   p-spher  
0        NaN  0.319697       NaN        NaN      NaN       NaN  
1   0.000003  0.259870  0.636965      False   0.4035  0.000133  
2        NaN  0.088017       NaN        NaN      NaN       NaN  
Required Sample Size per Group: 9.4155
Repeated ANOVA Result for condition comparison in adult mice:
        Source            SS  DF1  DF2            MS          F         p-unc  \
0    condition  18556.458500    1   28  18556.458500  35.770547  1.926847e-06   
1          day  11153.005396    4  112   2788.251349  15.625801  3.456488e-10   
2  Interact

In [9]:
results7 = tools.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='sex',
    subject='cohort_id'
)

print("Repeated ANOVA Result for sex comparison:")
print(results7['anova_result'])
print(f"Required Sample Size per Group: {results7['required_sample_size']:.4f}")

Repeated ANOVA Result for sex comparison:
        Source            SS  DF1  DF2           MS          F         p-unc  \
0          sex    591.269478    1   67   591.269478   0.625968  4.316300e-01   
1          day  13740.499208    4  268  3435.124802  26.555134  1.453509e-18   
2  Interaction   1913.458392    4  268   478.364598   3.697984  5.985023e-03   

      p-GG-corr       np2       eps sphericity   W-spher   p-spher  
0           NaN  0.009256       NaN        NaN       NaN       NaN  
1  2.967943e-14  0.283845  0.761674      False  0.541009  0.000005  
2           NaN  0.052307       NaN        NaN       NaN       NaN  
Required Sample Size per Group: 421.0108


In [10]:
male_merged_data = merged_data[merged_data['sex'] == 'male']
female_merged_data = merged_data[merged_data['sex'] == 'female']

tools8 = AnalysisTools(male_merged_data)
tools9 = AnalysisTools(female_merged_data)

results8 = tools8.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='condition',
    subject='cohort_id'
)

print("Repeated ANOVA Result for condition comparison in male mice:")
print(results8['anova_result'])
print(f"Required Sample Size per Group: {results8['required_sample_size']:.4f}")


results9 = tools9.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='condition',
    subject='cohort_id'
)

print("Repeated ANOVA Result for condition comparison in female mice:")
print(results9['anova_result'])
print(f"Required Sample Size per Group: {results9['required_sample_size']:.4f}")


Repeated ANOVA Result for condition comparison in male mice:
        Source            SS  DF1  DF2            MS          F         p-unc  \
0    condition  13803.361016    1   47  13803.361016  24.361437  1.045058e-05   
1          day   9766.531985    4  188   2441.632996  19.549159  1.812844e-13   
2  Interaction   1374.936737    4  188    343.734184   2.752139  2.946258e-02   

      p-GG-corr       np2       eps sphericity   W-spher   p-spher  
0           NaN  0.341381       NaN        NaN       NaN       NaN  
1  4.565038e-10  0.293755  0.721935      False  0.462933  0.000044  
2           NaN  0.055317       NaN        NaN       NaN       NaN  
Required Sample Size per Group: 8.6458
Repeated ANOVA Result for condition comparison in female mice:
        Source           SS  DF1  DF2           MS          F         p-unc  \
0    condition  8654.529426    1   18  8654.529426  10.972352  3.873324e-03   
1          day  5887.425614    4   72  1471.856403  12.145284  1.343875e-07   

In [11]:
results3 = tools3.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='sex',
    subject='cohort_id'
)

print("Repeated ANOVA Result for age comparison in SEFL:")
print(results3['anova_result'])
print(f"Required Sample Size per Group: {results3['required_sample_size']:.4f}")


results4 = tools4.repeated_measures_anova(
    dv='freezing_1_75s',
    within='day',
    between='sex',
    subject='cohort_id'
)

print("Repeated ANOVA Result for age comparison in control:")
print(results4['anova_result'])
print(f"Required Sample Size per Group: {results4['required_sample_size']:.4f}")


Repeated ANOVA Result for age comparison in SEFL:
        Source            SS  DF1  DF2           MS          F         p-unc  \
0          sex    633.829202    1   41   633.829202   0.683576  4.131441e-01   
1          day  13683.867112    4  164  3420.966778  20.184650  1.554356e-13   
2  Interaction   2457.270641    4  164   614.317660   3.624644  7.368865e-03   

      p-GG-corr       np2      eps sphericity   W-spher   p-spher  
0           NaN  0.016399      NaN        NaN       NaN       NaN  
1  1.540821e-10  0.329897  0.77897      False  0.530311  0.002362  
2           NaN  0.081225      NaN        NaN       NaN       NaN  
Required Sample Size per Group: 236.3465
Repeated ANOVA Result for age comparison in control:
        Source           SS  DF1  DF2          MS          F         p-unc  \
0          sex     3.325023    1   24    3.325023   0.028379  8.676315e-01   
1          day  1910.432112    4   96  477.608028  10.394922  4.885745e-07   
2  Interaction    64.238215  